# Importing libraries

In [ ]:
!pip install underthesea
!pip install pyvi
!pip install langdetect googletrans-py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=d9861679b7616357429b756cc5b19107f55f54d27739012ad2c39a62c8e752bf
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
  Created wheel for googletrans-py: filename=googletrans_py-4.0.0-py3-none-any.whl size=13115 sha256=38ca723b1b7cc48e457cc587452e0492118e58ce16bc7f9ac53904f07c39e183
  Stored in directory: /root/.cache/pip/wheels/e7/0d/ab/e65238ae958515ff99eb10e9f7d9de198b79

In [ ]:
import pandas as pd
import numpy as np
import re
from underthesea import text_normalize
from pyvi import ViTokenizer
from tqdm import tqdm
import time
from langdetect import detect, LangDetectException
from googletrans import Translator


tqdm.pandas()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Read data

## Candidate data

In [ ]:
file_path = '/content/drive/Shared drives/PTDL WEB/PTDL WEB - FINAL/Dataset/Cleaned data/cleaned_candidate_data.csv'

try:
    df_candidate = pd.read_csv(file_path, index_col=0)
    display(df_candidate.head())
except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy tệp tại đường dẫn {file_path}. Vui lòng kiểm tra lại đường dẫn và tên tệp.")
except Exception as e:
    print(f"Đã xảy ra lỗi khi đọc tệp: {e}")

,user_url,userid,user_name,industry,workplace_desired,desired_salary,gender,marriage,age,work_experience,desired_job_translated
0,https://timviec365.vn/uvtv2/dao-huu-tai-uv1111...,1111684868,Đào Hữu Tài,Kd Bất Động Sản,Hà Nội,Thỏa Thuận,Nam,Độc Thân,38.0,Dưới 1 năm,Chuyên Viên Kinh Doanh Bất Động Sản
1,https://timviec365.vn/uvtv2/ho-vinh-duc-uv199803,199803,Hồ Vĩnh Đức,Xây Dựng,Hồ Chí Minh,Thỏa Thuận,Nam,Khác,37.0,Không yêu cầu kinh nghiệm,Kỹ Sư Kinh Tế Xây Dựng
2,https://timviec365.vn/uvtv2/ngo-thi-bich-tram-...,1271594,Ngô Thị Bích Trâm,Khách Sạn - Nhà Hàng,Khánh Hòa,Thỏa Thuận,Nam,Độc Thân,NaN,Chưa Có Kinh Nghiệm,Nhân Viên Nhà Hàng
3,https://timviec365.vn/uvtv2/vo-ba-loi-uv160413,160413,Võ Bá Lợi,Giao Thông Vận Tải - Thủy Lợi - Cầu Đường,Hồ Chí Minh,Thỏa Thuận,Nam,Độc Thân,NaN,Chưa Có Kinh Nghiệm,Đầu Bếp
4,https://timviec365.vn/uvtv2/vo-thi-thuy-van-uv...,1111646039,Võ Thị Thuỳ Vân,Quản Trị Kinh Doanh,Đồng Nai,Thỏa Thuận,Nữ,Độc Thân,21.0,Chưa Có Kinh Nghiệm,Thực Tập Sinh Kinh Doanh


## Job data

In [ ]:
file_path = '/content/drive/Shared drives/PTDL WEB/PTDL WEB - FINAL/Dataset/Cleaned data/cleaned_job_data.csv'

try:
    df_job = pd.read_csv(file_path, index_col=0)
    display(df_job.head())
except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy tệp tại đường dẫn {file_path}. Vui lòng kiểm tra lại đường dẫn và tên tệp.")
except Exception as e:
    print(f"Đã xảy ra lỗi khi đọc tệp: {e}")

,Job_Name,Company_name,Location,Salary,Time,Benefit,Description,Requirement,URL,Industry,Type
0,Chuyên Viên Hỗ Trợ Kỹ Thuật ( ô tô thương mại)...,CÔNG TY CP TM DV AN SƯƠNG,Hà Nội,Cạnh tranh,10/31/2025,"Chế độ bảo hiểm, Du Lịch, Đồng phục, Chế độ th...",• Hỗ trợ kỹ thuật & hướng dẫn sửa chữa cho hệ ...,• Tốt nghiệp đại học ngành Cơ khí ô tô. • Tối ...,https://careerviet.vn/vi/tim-viec-lam/chuyen-v...,Cơ khí / Ô tô / Tự động hóa,Nhân viên chính thức
1,CHUYÊN VIÊN PHỤ TÙNG (MỚI),CÔNG TY CP TM DV AN SƯƠNG,Hồ Chí Minh,"5,000,000 - 10,000,000",9/30/2025,"Chế độ bảo hiểm, Du Lịch, Đồng phục, Chế độ th...","- Xây dựng chính sách, kế hoạch và KPI kinh do...","-Tốt nghiệp ĐH ngành Cơ khí ô tô, Logistics, K...",https://careerviet.vn/vi/tim-viec-lam/chuyen-v...,Tiếp thị / Marketing,Nhân viên chính thức
2,PHÓ TRƯỞNG BỘ PHẬN PHÁP LÝ – ĐĂNG KIỂM XE CBU/...,CÔNG TY CP TM DV AN SƯƠNG,Hà Nội,Cạnh tranh,8/31/2025,"Chế độ bảo hiểm, Du Lịch, Đồng phục, Chế độ th...",Mô tả công việc: - Quản lý hồ sơ pháp lý – kỹ ...,"- Tốt nghiệp kỹ thuật ô tô, cơ khí, luật hoặc ...",https://careerviet.vn/vi/tim-viec-lam/pho-truo...,Luật / Pháp lý,Nhân viên chính thức
3,Tài Xế Lái Xe Cho Giám Đốc (MỚI),CÔNG TY CP TM DV AN SƯƠNG,Hồ Chí Minh,"5,000,000 - 10,000,000",6/30/2025,"Chế độ bảo hiểm, Du Lịch, Đồng phục, Chế độ th...","Mô tả công việc Phụ trách đưa đón, chuyên chở ...","Nam, sức khỏe tốt, không bị cận; GPLX hạng B2 ...",https://careerviet.vn/vi/tim-viec-lam/tai-xe-l...,Vận chuyển / Giao nhận / Kho vận,Nhân viên chính thức
4,Phiên dịch viên tiếng Trung Quốc (MỚI),CÔNG TY CP TM DV AN SƯƠNG,Hà Nội,"15,000,000 - 20,000,000",8/31/2025,"Chế độ bảo hiểm, Du Lịch, Chế độ thưởng, Chăm ...","- Phiên dịch, biên dịch tài liệu kỹ thuật và n...",- Thành thạo tiếng Trung - Ưu tiên ứng viên có...,https://careerviet.vn/vi/tim-viec-lam/phien-di...,Hành chính / Thư ký,Nhân viên chính thức


# Feature Engineering


In [ ]:
#Ký tự đặc biệt
_PUNCTS = [
    '"', ')', '(', '-', '|', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#',
    '*', '+', '\\', '•', '~', '@', '£', '·', '{', '}', '©', '^', '®', '`',
    '<', '→', '°', '€', '™', '›', '♥', '←', '×', '§', '″', '′', 'Â', '█',
    '½', 'à', '…', '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬',
    '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹',
    '─', '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫',
    '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '∙',
    '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣',
    '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', '.', ',', '!', '?', ':' , ';'
]

#Danh sách stopwords
vietnamese_stopwords_list = [
    'và', 'là', 'của', 'có', 'được', 'cho', 'thì', 'ở', 'tại', 'trong', 'ra', 'với', 'khi', 'để', 'bị', 'đã', 'về',
    'này', 'một', 'cũng', 'không', 'vào', 'đến', 'từ', 'các', 'như', 'làm', 'phải', 'đây', 'đó', 'việc', 'trên', 'qua',
    'lại', 'hay', 'theo', 'cần', 'chỉ', 'tới', 'sẽ', 'hơn', 'sau', 'trước', 'nhất', 'rất', 'luôn', 'mỗi', 'hoặc',
    'khác', 'yêu cầu', 'công việc', 'mô tả', 'phúc lợi', 'quyền lợi', 'chi tiết'
]


In [ ]:
def preprocess_text_for_tfidf_word2vec(text_series: pd.Series, min_token_len: int = 2) -> pd.Series:
    """
    Comprehensive preprocessing for a text Series, suitable for TF-IDF and Word2Vec.
    """

    # This inner function processes each individual text string
    def process_single_text(text_element: str) -> str:
        # Initialize processed_text to ensure it's always defined
        processed_text = ""

        if pd.isna(text_element):
            processed_text = "" # Explicitly set to empty string for NaN/None
        else:
            try:
                processed_text = str(text_element) # Convert to string
            except Exception:
                # If conversion to string fails, it remains an empty string
                pass

        # Subsequent processing steps
        try:
            # 2. Normalize text (e.g., Unicode normalization for Vietnamese)
            processed_text = text_normalize(processed_text)

            # 3. Convert to lowercase
            processed_text = processed_text.lower()

            # 4. Replace punctuation with spaces
            punct_to_space = str.maketrans({char: ' ' for char in _PUNCTS})
            processed_text = processed_text.translate(punct_to_space)

            # 5. Vietnamese Word Tokenization (Segmentation)
            processed_text = ViTokenizer.tokenize(processed_text)

            # 6. Remove extra whitespace
            processed_text = re.sub(r'\s+', ' ', processed_text).strip()

        except Exception as e:
            pass

        return processed_text

    # Apply the inner function to each element of the input Series
    return text_series.apply(process_single_text)
def apply_preprocessing_to_series(text_series: pd.Series, min_token_len: int = 2) -> pd.Series:
    return text_series.apply(lambda x: preprocess_text_for_tfidf_word2vec(pd.Series([x]), min_token_len=min_token_len).iloc[0] if pd.notna(x) else "")

In [ ]:
candidate_text_cols = {
    'desired_job_translated': 'proc_desired_job',
    'industry': 'proc_industry',
    'workplace_desired': 'proc_workplace',
}

In [ ]:
for original_col, new_col_name in candidate_text_cols.items():
  df_candidate[new_col_name] = apply_preprocessing_to_series(df_candidate[original_col])

In [ ]:
df_candidate.to_csv("prcocess_candidate.csv")

## Normalizing candidate

In [ ]:
candidate_text_cols = {
    'desired_job_translated': 'proc_desired_job',
    'industry': 'proc_industry',
    'workplace_desired': 'proc_workplace',
}

In [ ]:
display(df_candidate.head())

## Normalizing Job

In [ ]:
def translate_with_retry(text, target_lang='vi', retries=3):
    """
    Translates text with an automatic retry mechanism to handle errors.
    """
    # Initialize a new translator for each call to avoid connection issues
    translator = Translator()
    for i in range(retries):
        try:
            # Detect language
            lang = detect(text)
            # Only translate if it's not already in the target language
            if lang != target_lang:
                return translator.translate(text, src=lang, dest=target_lang).text
            else:
                return text # Return original text if it's already Vietnamese
        except Exception as e:
            # If it's the last attempt, print the error and return the original text
            if i == retries - 1:
                # print(f"Translation error after {retries} attempts: {e}. Using original text.")
                return text
            # Wait a moment before retrying
            time.sleep(1.5 ** i) # Exponential backoff: 1s, 1.5s, 2.25s


def process_vietnamese_text(text):
    """
    This function only performs preprocessing for Vietnamese text.
    """
    # Convert to lowercase
    text = text.lower()
    # Replace punctuation with spaces
    punct_to_space = str.maketrans({char: ' ' for char in _PUNCTS})
    text = text.translate(punct_to_space)
    # Remove special characters, numbers, and punctuation
    text = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệíìỉĩịóòỏõọôốồổỗộơớờởỡợúùủũụưứừửữựýỳỷỹỵđ]', ' ', text)
    # Tokenize Vietnamese text
    text = ViTokenizer.tokenize(text)
    # Remove stopwords
    words = [word for word in text.split() if word not in vietnamese_stopwords_list]

    return ' '.join(words)


def translate_and_process_pipeline(text):
    """
    Complete pipeline: Translate first, then preprocess.
    """
    text = str(text)
    if text.lower() == 'nan' or not text.strip():
        return ""

    # Step 1: Translate text with the retry mechanism
    translated_text = translate_with_retry(text)

    # Step 2: Preprocess the translated Vietnamese text
    return process_vietnamese_text(translated_text)

In [ ]:
columns_to_process = ['Job_Name','Location', 'Description', 'Requirement', 'Industry', 'Type']

print("Starting the text translation and preprocessing...")
print("Note: This process may take a long time due to translation API calls.")

for col in columns_to_process:
    if col in df_job.columns:
        print(f"--- Processing column: {col} ---")
        processed_col_name = f"{col}_processed"

        # FIX: Replace inplace=True by reassigning the value
        df_job[col] = df_job[col].fillna('')

        # Apply the complete pipeline function
        df_job[processed_col_name] = df_job[col].progress_apply(translate_and_process_pipeline)
    else:
        print(f"Warning: Column '{col}' not found in the DataFrame.")

print("\nProcess completed.")

Starting the text translation and preprocessing...
Note: This process may take a long time due to translation API calls.
--- Processing column: Job_Name ---


100%|██████████| 21858/21858 [1:08:05<00:00,  5.35it/s]


Process completed.


In [ ]:
from google.colab import files

df_job.to_csv('df_job.csv', index=False)
files.download('df_job.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>